In [16]:
#get dataframe of tweet_id and user_id
import pandas
import csv
from twython import Twython
import time
from collections import defaultdict

df = pandas.read_csv('gamergate_tweets.csv')
user_ids = df['user_id'].tolist()[:5000]

# Consumer & Auth keys and tokens for authentication
CONSUMER_KEY = "48r56upQvsnTiSDfY4CNwIpCQ"
CONSUMER_SECRET = "VqZhFW7GqogdKXJnpuS0umVe211bFgoCdaQlRdRMayvrbdDh3k"
OAUTH_TOKEN = "18000528-Scl8a1pLinqP4KWtaa3f8EN0M1vqAGTyYXMtsLIGm"
OAUTH_TOKEN_SECRET = "NcIvLeqVbwwBegabvo7946FVIaDN0CAaJ7aS2aV7pZFzC"

twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

# check rate limit
# twitter.get_application_rate_limit_status()['resources']['followers']

In [47]:
# GET followers/list
def get_follower_list(twitter, user_id):
    followers = []
    cursored = twitter.cursor(twitter.get_followers_list, id=user_id)
    for follower in cursored:
        followers.append(follower)
    return followers

# GET users/lookup
def get_user_lookup(twitter, current):
    user_obj = twitter.lookup_user(user_id=current)
    return user_obj
    
# GET followers/ids
def get_follower_ids(twitter, user_id):
    followers = []
    cursored = twitter.cursor(twitter.get_followers_ids, id=user_id)
    for follower in cursored:
        followers.append(follower)
    return followers

In [ ]:
user_list = []
start = 0
end = 100
while end <= 5000:
    current = user_ids[start:end]
    start += 100
    end += 100
    users = get_user_lookup(twitter, current)
    for i in range(0, len(users)):
        each_user = []
        each_user.append(user_ids[i]) # user[0] = user_id
        each_user.append(users[i]['followers_count']) # user[1] = follower count
        each_user.append(users[i]["favourites_count"])
        user_list.append(each_user)

In [18]:
users_df = pandas.DataFrame(user_list, columns=["User ID", "Follower Count", "Favourites Count"])
                
#print users_df.sort_values(["Follower Count", "Favourites Count",], ascending=[False, False])

In [61]:
# popular user_id list:
#           [-4, -4, -4, -4, -4, 3, ]
pop_users = [1498976455, 16088388, 2368624716, 1317712621, 2801289353, 
             2756744663, 857720059, 24610050, 2790717530, 854203626, 115601411,
             2788941283, 2806358985, 47622504, 33128909, 2837175356, 104519838, 
             34283550, 1499808391, 356199261, 574962273]

follower_ids = defaultdict(list)
count = 0
remaining = twitter.get_application_rate_limit_status()['resources']['followers']['/followers/ids']['remaining']
for user in pop_users:
    if remaining <= 1:
        print "start waiting"
        time.sleep(1000)
        print "done waiting"
        remaining = twitter.get_application_rate_limit_status()['resources']['followers']['/followers/ids']['remaining']
        print remaining
    else:
        follower_ids[user] = get_follower_ids(twitter, user)
        count += 1
        remaining = twitter.get_application_rate_limit_status()['resources']['followers']['/followers/ids']['remaining']
        print remaining

14
13
11
10
9
8
7
3
2
1
start waiting
done waiting
15
14
13
12
11
10
9
8
6
5
4
0 []


In [79]:
# TODO follower_id is missing one user, 
# I added it manuelly right now b/c I'm being lazy 
# but this need to be fixed later
print follower_ids.keys()

[574962273, 24610050, 2788941283, 16088388, 1498976455, 47622504, 2801289353, 854203626, 1499808391, 2368624716, 1317712621, 33128909, 2806358985, 115601411, 34283550, 2756744663, 2790717530, 857720059, 2837175356, 356199261, 104519838]


[3419735987, 710100971003768833, 1965525516, 4803793910, 709747303054229504, 4703173789, 703763643054333953, 3352612113, 19865528, 702892070290202624, 15088481, 252219542, 1273110458, 4878766583, 10515212, 2953926716, 2406394634, 3328531330, 2786395269, 4259676672, 2916604288, 3107110095, 33449297, 3109028075, 4882553421, 4285199747, 4837806013, 2569588634, 228676502, 1486379629, 11353012, 4831259062, 1903362108, 3009483972, 4749608450, 4610848637, 3036878181, 4015682026, 2751259861, 3375443631, 3663894253, 3107721071, 3000160524, 4449537796, 536950156, 247613774, 1730296904, 132789518, 2289899511, 4205720293, 2833599255, 1950799526, 3364725173, 459520912, 3253962836, 4085687661, 1055786635, 2915348348, 4076264120, 4064949138, 3098086860, 19351599, 3099971355, 1954327123, 41201457, 24774382, 1119578671, 3066206409, 2746258501, 2359938182, 2802062539, 156285331, 392078103, 2747746256, 3106787658, 19290107, 275653130, 21245956, 23002858, 2235210511, 3142647066, 3328719406, 92909669, 3633

1498976455
16088388
2368624716
1317712621
2801289353
2756744663
857720059
24610050
2790717530
854203626
115601411
2788941283
2806358985
47622504
33128909
2837175356
104519838
34283550
1499808391
356199261
574962273
0
